<a href="https://colab.research.google.com/github/biggymart/colab/blob/main/autokeras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**오토케라스**

**들어가며**

딥러닝에는 다양한 모델이 있는데, 그걸 다 손으로 일일히 확인해가는 일은 time-consuming할 뿐만 아니라 힘도 무지하게 든다. 다행히도 2020년도 초반에 나온 오토케라스 라이브러리를 활용하면 이 과정을 자동으로 할 수 있다.

sci-kit learn처럼 모듈화되어 있어서 매우 짧은 코드로 많은 작업을 할 수 있다. 오토케라스에 대한 구체적인 설명 및 튜토리얼은 다음의 링크를 통해 확인하도록 하자:

https://autokeras.com/tutorial/overview/

본 문서는 오토케라스의 구체적인 내용은 다루지 않는다. 이 라이브러리를 당장 사용해보고 싶은 독자를 대상으로 작성되었다.

---

**목차**


1.   autokeras 라이브러리 설치
2.   autokeras 실행
3.   모델 구조 확인 및 저장

---

NOTE> 코드 블럭을 확인할 때 `<change>`로 주석이 되어 있는 부분을 주목해서 읽도록 하자.

#**1. autokeras 라이브러리 설치**

**(1) .py 파일에서 작업하는 경우**

**cmd**창에서 pip을 통해 autokeras를 설치한다:

`pip install autokeras`

성공적으로 다운로드 됐다면 다음과 같은 문구가 보인다:

`Successfully installed autokeras-1.0.12 keras-tuner-1.0.2 tabulate-0.8.9 tensorflow-2.4.1 terminaltables-3.1.0`

**(2) .ipynb 파일에서 작업하는 경우**

`!pip install autokeras` 통해 autokeras를 설치하자.

cf> 본 문서는 colab에서 ipynb 파일을 기준으로 작성되었다.


#**2. autokeras 실행**

아래의 코드 블럭은 가장 간단한 형태의 모델이다.

modeling 부분에서 주목할 점은 기존의 모델링과 달리 input shape을 명시하지 않고, 심지어 y 데이터를 One Hot Encoding하지 않아도 오류 없이 모델이 돌아간다는 부분이다. 

하이퍼파라미터 튜닝이라고는 할 것이 고작 max_trials를 조정하고 fit의 epochs를 조정하는 수준이다. 매우 간단하다.

주의: 여러 가지 모델들을 실행하여 시간이 많이 소요되므로 max_trials와 epochs의 크기는 작게 설정하였다.

In [ ]:
# autokeras101

!pip install autokeras
import numpy as np
import tensorflow as tf
import autokeras as ak
from tensorflow.keras.datasets import mnist


# 1. load data
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(60000, 28, 28, 1).astype('float32')/255.
x_test = x_test.reshape(10000, 28, 28, 1).astype('float32')/255.

# 2. modeling
model = ak.ImageClassifier(
    overwrite = True, # Input shape도 없고, OneHotEncoding 안 해도 돌아감
    max_trials = 2  # 튜닝할 게 있다면 유일하게 이 부분
)

# 3. fit
model.fit(x_train, y_train, epochs=3)

# 4. evalute
results = model.evaluate(x_test, y_test)
print(results)

Trial 2 Complete [00h 25m 50s]
val_loss: 0.2786787450313568

Best val_loss So Far: 0.04720625653862953
Total elapsed time: 00h 26m 13s
INFO:tensorflow:Oracle triggered exit
Epoch 1/3
1875/1875 [==============================] - 7s 4ms/step - loss: 0.2982 - accuracy: 0.9081
Epoch 2/3
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0788 - accuracy: 0.9760
Epoch 3/3
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0639 - accuracy: 0.9811
INFO:tensorflow:Assets written to: ./image_classifier/best_model/assets
313/313 [==============================] - 1s 3ms/step - loss: 0.0494 - accuracy: 0.9839
[0.04941147565841675, 0.9839000105857849]


겨우 3 epochs만 수행했는데도 accuracy가 상당히 좋다. (필자의 경우, loss = 0.0494; acc = 0.9839 정도가 나왔다)

일단 기본적인 구성은 알았으니, 모델 구색을 갖춰보도록 하자. 위 `autokeras101` 코드에 (1) OneHotEncoding (2) loss 및 metrics 명시 (3) callbacks 추가 해보도록 하겠다.

주의: 본 문서에서 `EarlyStopping` 및 `ReduceLROnPlateau`에서 설정된 `patience` 값이 `epochs` 보다 크거나 같기 때문에 정상적으로 콜백이 작동하지 않는다. 여기서는 그저 이런 옵션을 넣을 수 있다는 점을 보여주기 위해서 콜백을 포함시켰을 뿐이다.

In [ ]:
# autokeras102

import numpy as np
import tensorflow as tf
import autokeras as ak
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint,\
    ReduceLROnPlateau

(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(60000, 28, 28, 1).astype('float32')/255.
x_test = x_test.reshape(10000, 28, 28, 1).astype('float32')/255.

### <change_1> ###
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
### </change_1> ###

model = ak.ImageClassifier(
    overwrite=True,
    max_trials=2,
    ### <change_2> ###
    loss='mse',
    metrics=['acc']
    ### </change_2> ###
)

### <change_3> ###
es = EarlyStopping(monitor='val_loss', mode='min', patience=6)
lr = ReduceLROnPlateau(monitor='val_loss', patience=3, factor=0.5, verbose=2)
ck = ModelCheckpoint('./temp/', save_best_only=True,
    save_weights_only=True, monitor='val_loss', verbose=1
)
### </change_3> ###

model.fit(x_train, y_train, epochs=3, validation_split=0.2, 
    callbacks=[es, lr, ck]
)

results = model.evaluate(x_test, y_test)
print(results)

#**3. 모델 구조 확인 및 저장**

그런데 이렇게 만들어 놓고 보니, 모델의 구조를 확인하거나 모델을 저장해놓고 사용하면 정말 좋을 것 같다. 그래서 중간에 `model.summary()` 라던가 `model.save('path.h5')`를 사용해보면 어떨까?

하지만 뜻밖에도 다음과 같은 에러를 보게 된다:

`AttributeError: 'ImageClassifier' object has no attribute 'summary'`

`AttributeError: 'ImageClassifier' object has no attribute 'save'`

오토케라스의 모델을 h5 파일로 저장하거나 summary를 보기 위해선 먼저 다음과 같이 모델을 "내보내야"(export) 한다:

`model.export_model()`

In [ ]:
# autokeras103

import numpy as np
import tensorflow as tf
import autokeras as ak
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint,\
    ReduceLROnPlateau

(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(60000, 28, 28, 1).astype('float32')/255.
x_test = x_test.reshape(10000, 28, 28, 1).astype('float32')/255.

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

model = ak.ImageClassifier(
    overwrite=True,
    max_trials=2,
    loss='mse',
    metrics=['acc']
)

es = EarlyStopping(monitor='val_loss', mode='min', patience=6)
lr = ReduceLROnPlateau(monitor='val_loss', patience=3, factor=0.5, verbose=2)
ck = ModelCheckpoint('./temp/', save_best_only=True,
    save_weights_only=True, monitor='val_loss', verbose=1
)

model.fit(x_train, y_train, epochs=3, validation_split=0.2, 
    callbacks=[es, lr, ck]
)

results = model.evaluate(x_test, y_test)
print(results)

### <change_1> ###
model2 = model.export_model()
model2.save('path/to/save/model.h5')
model2.summary()
### </change_1> ###

### <change_2> ###
best_model = model.tuner.get_best_model()
best_model.save('path/to/save/best_model.h5')
### </change_2> ###

기존 `model` 모델과 구분하기 위해 변수명을 `model2`로 설정하였다. 위 코드를 실행하면 `model2`에 대해서 정상적으로 저장하고 요약을 출력해주는 걸 확인할 수 있다.


---


한편, 모델을 저장하긴 하는데 도대체 어떤 모델을 저장했는지에 대해서 궁금하다. 사실, `model.export_model()`을 통해서 내보내지는 모델은 여러 모델 중 가장 좋은 모델이다. `model.tuner.get_best_model()`을 통해 가장 좋은 모델을 저장하고 이전에 저장했던 모델과 비교해보면 이 사실을 알 수 있다.

두 모델 비교는 아래 코드에서처럼 summary를 찍어보면 확인할 수 있다:

In [ ]:
# keras104

import numpy as np
import tensorflow as tf
import autokeras as ak
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint,\
    ReduceLROnPlateau
from tensorflow.keras.models import load_model

(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(60000, 28, 28, 1).astype('float32')/255.
x_test = x_test.reshape(10000, 28, 28, 1).astype('float32')/255.

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

### <change_1> ###
model = load_model('path/to/save/model.h5')
model.summary()

best_model = load_model('path/to/save/best_model.h5')
best_model.summary()

results = model.evaluate(x_test, y_test)
print(results)

best_results = best_model.evaluate(x_test, y_test)
print(best_results)
### </change_1> ###

**마치며**

위와 같이 초보자도 쉽게 사용할 수 있는 오토케라스의 활용법을 알아보았다. 짧은 코드로 많은 작업을 할 수 있는 "갓성비" 오토케라스를 많이 활용해보자.